# Scraping the text

In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

visitedLinks = set()

In [6]:
def CrawlWebsite(url, baseUrl, maxDepth=1, currentDepth=0):
    if currentDepth > maxDepth or url in visitedLinks:
        return

    visitedLinks.add(url)

    # Get the webpage content
    try:
        response = requests.get(url)
        content_type = response.headers.get('content-type')

        if 'application/pdf' not in content_type:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract text from the page
            pageText = soup.get_text(separator=' ', strip=True) #could use (separator=' ', strip=True) to have already parsed text
            print(f"URL: {url}")
            #print(f"Text:\n{pageText[:500]}...")  # Display first 500 characters
            with open('Dumps/restaurant.txt', 'a') as f:
              f.write(pageText)
              f.close()
            print("\n\n")

        # Find all internal links
        for link in soup.find_all('a', href=True):
            href = link['href']
            parsedHref = urlparse(href)

            # Check if the link is an internal link
            if not parsedHref.netloc or parsedHref.netloc == urlparse(baseUrl).netloc:
                newUrl = urljoin(baseUrl, href)
                CrawlWebsite(newUrl, baseUrl, maxDepth, currentDepth + 1)

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve {url}: {e}")

    # Pause to avoid overwhelming the server
    time.sleep(0.1)

In [7]:
startUrl = "https://www.pacificpinestavern.com.au/"
baseUrl = startUrl

CrawlWebsite(startUrl, baseUrl)

URL: https://www.pacificpinestavern.com.au/



URL: https://www.pacificpinestavern.com.au/#home-banner-section



URL: https://www.pacificpinestavern.com.au/#home-quick-links-section



URL: https://www.pacificpinestavern.com.au/#home-featured-content-block-section



URL: https://www.pacificpinestavern.com.au/#home-core-content-section



URL: https://www.pacificpinestavern.com.au/#home-whats-on-section



URL: https://www.pacificpinestavern.com.au/#family-bistro-section



URL: https://www.pacificpinestavern.com.au/#new-page-section



URL: https://www.pacificpinestavern.com.au/#home-dining-2-section



URL: https://www.pacificpinestavern.com.au/#home-functions-section



URL: https://www.pacificpinestavern.com.au/#home-gallery-section



URL: https://www.pacificpinestavern.com.au/#home-sfmc-popup-section-section



URL: https://www.pacificpinestavern.com.au/dining#dining-room-1-section



URL: https://www.pacificpinestavern.com.au/dining#dining-room-2-section



URL: https://www.pac

UnboundLocalError: cannot access local variable 'soup' where it is not associated with a value

# Generating Word Embedding and Topic Modelling

In [13]:
import spacy
nlp = spacy.load('en_core_web_md')

In [15]:
with open('Dumps/restaurant.txt', "rt") as f:
    text = f.read()

print(text)

Pacific Pines Tavern, Pacific Pines, QLD Please enable JavaScript to use this page! For full functionality of this site it is necessary to enable  JavaScript. Here are the instructions how to enable JavaScript in your web browser . Home Banner Home Quick Links Home Featured Content block Home Core Content Home What's On Family Bistro New Page Finish Line Bar Home Functions Home Gallery Home SFMC Popup Section Home Eat Family Bistro FinishLine Bar Kids Our Menus Drink Our Bars FinishLine Bar Bistro Bar What's On Sport UFC pub+ Functions Our Rooms Function Pack Enquire Now Gaming What's On Events Bars Sport UFC Jobs Contact Contact Us Subscribe Book Now Home Banner Home Quick Links Home Featured Content block Home Core Content Home What's On Family Bistro New Page Finish Line Bar Home Functions Home Gallery Home SFMC Popup Section Home Eat Family Bistro FinishLine Bar Kids Our Menus Drink Our Bars FinishLine Bar Bistro Bar What's On Sport UFC pub+ Functions Our Rooms Function Pack Enquir

In [17]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

Pacific Pines Tavern ORG
Pacific Pines LOC
QLD GPE
Banner Home Quick Links Home Featured Content ORG
Home Core Content Home ORG
Family Bistro ORG
Sport UFC pub+ Functions PRODUCT
Home Core Content Home ORG
Family Bistro ORG
Sport UFC pub+ Functions PRODUCT
Central Park LOC
Hotham Drive FAC
the Pacific Pines Central Park FAC
Pacific Pines Tavern FAC
Pacific Pines LOC
the Gold Coast LOC
the Pacific Pines Central Park FAC
Family Bistro ORG
Central Park LOC
Finish Line Bar ORG
150 inch QUANTITY
every minute TIME
one CARDINAL
ON Family Bistro Our Bistro ORG
every day of the week DATE
the Gold Coast LOC
Bistro ORG
11:30 am TIME
one CARDINAL
Gold Coast's ORG
150 inch QUANTITY
two CARDINAL
TAB ORG
two CARDINAL
two CARDINAL
Finishline Bar ORG
a long day DATE
the weekend DATE
The Finishline Bar FAC
Pacific Pines Tavern FAC
Sports Bar ORG
8 CARDINAL
Hotham Drive GPE
Pacific Pines LOC
QLD GPE
Australia GPE
07 CARDINAL
5580 6100 MONEY
Pacific Pines Tavern ORG
Hotham Dve Pacific Pines ORG
QLD GPE
07

In [18]:
from spacy import displacy

displacy.render(doc, style="ent")

In [26]:
with open("NER/ner_results.txt", 'a') as f:
    for ent in doc.ents:
        f.write(f"{ent.text},{ent.label_}\n")
    f.close()

# ChatGPT Generation of Questions

In [29]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv